In [72]:
#ライブラリをインポート
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from ydata_profiling import ProfileReport

print("Importが完了しました")

Importが完了しました


In [73]:
path = "/kaggle/input/playground-series-s5e10/"
train_df = pd.read_csv(path + "train.csv")
test_df = pd.read_csv(path + "test.csv")

df = pd.concat([train_df, test_df],axis=0)
df.tail()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
172580,690334,rural,2,0.01,45,dim,rainy,False,False,afternoon,True,True,2,NaN
172581,690335,rural,1,0.74,70,daylight,foggy,False,True,afternoon,False,False,2,NaN
172582,690336,urban,2,0.14,70,dim,clear,False,False,evening,True,True,1,NaN
172583,690337,urban,1,0.09,45,daylight,foggy,True,True,morning,False,True,0,NaN
172584,690338,highway,1,0.63,35,night,foggy,True,False,evening,False,False,0,NaN


In [75]:
df['Condition_01'] = df['lighting'] + '_'+df['weather']
#df['Condition_02'] = df['lighting'] + '_'+df['weather'] + '_'+df['time_of_day']
print(f"----------Condition01----------\n{df['Condition_01'].value_counts()}")

df['Condition_03'] = 'normal'
df.loc[df['Condition_01'].isin(['night_foggy','night_rainy']),'Condition_03']='bad'
#print(df['Condition_02'].value_counts())
print(f"----------Condition03----------\n{df['Condition_03'].value_counts()}")

----------Condition01----------
Condition_01
dim_foggy         87986
daylight_foggy    84625
night_clear       82713
dim_rainy         78551
dim_clear         78432
daylight_clear    78143
daylight_rainy    74644
night_foggy       69088
night_rainy       56157
Name: count, dtype: int64
----------Condition03----------
Condition_03
normal    565094
bad       125245
Name: count, dtype: int64


road_highway                 bool
road_rural                   bool
road_urban                   bool
light_daylight               bool
light_dim                    bool
light_night                  bool
weather_clear                bool
weather_foggy                bool
weather_rainy                bool
con3_bad                     bool
timeofday_afternoon          bool
timeofday_evening            bool
timeofday_morning            bool

In [76]:
#con1 = pd.get_dummies(df["Condition_01"],prefix = "con1")
#con2 = pd.get_dummies(df["Condition_02"],prefix = "con2")
con3 = pd.get_dummies(df["Condition_03"],prefix = "con3")
rt = pd.get_dummies(df["road_type"],prefix = "road")
#lt = pd.get_dummies(df["lighting"],prefix = "light")
#wt = pd.get_dummies(df["weather"],prefix = "weather")
#td = pd.get_dummies(df["time_of_day"],prefix = "timeofday")

#light,weather,timeofdayはラベルエンコする
lightmap ={
    "daylight": 1,
    "dim" : 2,
    "night" : 3
}
df["lighting_enc"] = df["lighting"].map(lightmap)

weathermap = {
    "foggy":3,
    "rainy":2,
    "clear":1
}
df["weather_enc"] = df["weather"].map(weathermap)

timemap = {
    "evening":3,
    "afternoon":2,
    "morning":1
}
df["time_enc"] = df["time_of_day"].map(timemap)

#light,weather,timeofdayから新しい特徴量を作成する
df["dng_score"] =  df["lighting_enc"]+df["weather_enc"]+df["time_enc"]

df["dng_score"].value_counts()
df.head(10)

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk,Condition_01,Condition_03,lighting_enc,weather_enc,time_enc,dng_score
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13,daylight_rainy,normal,1,2,2,5
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35,daylight_clear,normal,1,1,3,5
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30,dim_clear,normal,2,1,1,4
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21,dim_rainy,normal,2,2,1,5
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56,daylight_foggy,normal,1,3,3,7
5,5,highway,3,0.54,70,night,foggy,True,False,morning,False,True,0,0.61,night_foggy,bad,3,3,1,7
6,6,highway,1,0.04,45,night,foggy,False,True,morning,True,False,2,0.20,night_foggy,bad,3,3,1,7
7,7,highway,2,0.33,45,daylight,clear,False,True,evening,False,True,0,0.16,daylight_clear,normal,1,1,3,5
8,8,rural,3,0.63,25,dim,clear,True,False,evening,True,True,2,0.18,dim_clear,normal,2,1,3,6
9,9,urban,4,0.38,45,dim,clear,True,True,afternoon,True,True,2,0.15,dim_clear,normal,2,1,2,5


In [105]:
df2 = pd.concat([df, rt, con3],axis=1)
df3 = df2.drop(columns=["lighting","road_type","weather","time_of_day","Condition_01","Condition_03","con3_normal"])

bool_columns = [
    'road_highway',
    'road_rural',
    'road_urban',
    #'light_daylight',
    #'light_dim',
    #'light_night',
    #'weather_clear',
    #'weather_foggy',
    #'weather_rainy',
    'con3_bad',
    #'timeofday_afternoon',
    #'timeofday_evening',
    #'timeofday_morning',
    'road_signs_present',
    'public_road',
    'holiday',
    'school_season'
]

df3[bool_columns] = df3[bool_columns].astype(int)
df3["con3_bad"].value_counts()
df3.head()

,id,num_lanes,curvature,speed_limit,road_signs_present,public_road,holiday,school_season,num_reported_accidents,accident_risk,...,dng_score,trafic,students,road,slippy,visibility,road_highway,road_rural,road_urban,con3_bad
0,0,2,0.06,35,0,1,0,1,1,0.13,...,5,1,0,2,20.7936,1.67,0,0,1,0
1,1,4,0.99,35,1,0,1,1,0,0.35,...,5,1,1,1,5.4900,1.67,0,0,1,0
2,2,4,0.63,70,0,1,1,0,2,0.30,...,4,1,0,1,8.6300,1.33,0,1,0,0
3,3,4,0.07,35,1,1,0,0,1,0.21,...,5,2,0,2,20.8849,1.67,1,0,0,0
4,4,1,0.58,60,0,0,1,0,1,0.56,...,7,1,0,1,7.5800,2.33,0,1,0,0


In [106]:
df3.dtypes

id                          int64
num_lanes                   int64
curvature                 float64
speed_limit                 int64
road_signs_present          int64
public_road                 int64
holiday                     int64
school_season               int64
num_reported_accidents      int64
accident_risk             float64
lighting_enc                int64
weather_enc                 int64
time_enc                    int64
dng_score                   int64
trafic                      int64
students                    int64
road                        int64
slippy                    float64
visibility                float64
road_highway                int64
road_rural                  int64
road_urban                  int64
con3_bad                    int64
dtype: object

In [107]:
#統合したデータを再分割。
train_df2 = df3[~df3["accident_risk"].isnull()]
test_df2 = df3[df3["accident_risk"].isnull()]
print("分割が完了しました")

分割が完了しました


In [108]:
#説明変数と目的変数を分割する。
train_x = train_df2.drop(columns=["accident_risk"])
train_y = train_df2["accident_risk"]

test_x = test_df2.drop(columns=["accident_risk"])
test_y = test_df2["accident_risk"]

print("分割が完了しました")

分割が完了しました


In [ ]:
#categories = ["road_type", "lighting", "weather", "time_of_day"]

# XGBoost パラメータチートシート（回帰版）

「過学習抑制」「汎化性能向上」「高速化」にフォーカスした実務向けまとめです。

------------------------------------------------------------
## 0. 目的関数・評価指標（回帰）

- objective:
  - reg:squarederror  … 一般的な回帰（MSE）
  - count:poisson     … カウントデータの回帰（非負整数・対数リンク）
- eval_metric（学習時の監視指標の例）:
  - rmse, mae, rmsle など（複数指定可）

※ 目的関数は通常「reg:squarederror」で問題ありません。
※ 最終評価は RMSE/MAE/R² を報告するのが定番です。

------------------------------------------------------------
## 1. 木の構造制御（過学習対策の要）

- max_depth（木の最大深さ）
  - 目安: 3〜10
  - 小さいほど過学習しにくいが、浅すぎると表現力不足

- min_child_weight（葉に必要な最小サンプル重みの合計）
  - 目安: 1〜10
  - 大きくすると分割が慎重になり過学習抑制

- gamma（min_split_loss / 分割で必要な最小損失減少）
  - 目安: 0〜5
  - 大きいほど分割を抑制 → 過学習抑制

------------------------------------------------------------
## 2. サンプリング（汎化性能アップ）

- subsample（行サンプリング率）
  - 目安: 0.6〜0.9
  - 小さくすると汎化向上・過学習抑制、下げ過ぎると精度低下

- colsample_bytree（列サンプリング率）
  - 目安: 0.6〜0.9
  - 高次元で有効、過学習抑制

（補足）
- colsample_bylevel / colsample_bynode も細かい列サンプリング制御に使えますが、まずは colsample_bytree を優先。

------------------------------------------------------------
## 3. 学習率とブースト制御

- learning_rate（学習率）
  - 目安: 0.01〜0.1
  - 小さいほど汎化が良くなりやすい → 代わりに n_estimators を増やす

- n_estimators（木の本数）
  - 目安: 100〜2000
  - early_stopping_rounds と併用して最適本数を自動調整

------------------------------------------------------------
## 4. 正則化（過学習抑制）

- reg_lambda（L2 正則化 / 旧名: lambda）
  - 目安: 1〜10
  - デフォルト 1、過学習時は増やす

- reg_alpha（L1 正則化 / 旧名: alpha）
  - 目安: 0〜10
  - スパース性が高い場合に有効（特徴選択的効果）

------------------------------------------------------------
## 5. その他の重要設定

- tree_method
  - hist       … CPUで高速。まずはこれ
  - gpu_hist   … GPUがあるなら最速

- early_stopping_rounds
  - 例: 50〜200
  - 検証データの指標が改善しなくなったら学習打ち切り

- random_state（再現性） / n_jobs（並列数）
  - 再現性確保と計算資源の調整に有用

- enable_categorical（カテゴリを整数化して使う場合）
  - バージョン対応前提。多値カテゴリは one-hot よりこちらが有利な場合あり
  - 二値は 0/1 数値列のままで十分

------------------------------------------------------------
## 6. 過学習を防ぐための基本セット（例：XGBRegressor）

```python
from xgboost import XGBRegressor

model = XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    tree_method='hist',        # GPUなら 'gpu_hist'
    n_estimators=1000,         # 多め + 早期打ち切りで最適化
    learning_rate=0.05,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=2.0,
    reg_alpha=1.0,
    random_state=42
)
# model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=100, verbose=False)

In [110]:
#モデルの呼び出し
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    tree_method='hist',        # GPUなら 'gpu_hist'
    n_estimators=1000,         # 多め + 早期打ち切りで最適化
    learning_rate=0.05,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=2.0,
    reg_alpha=1.0,
    random_state=42
)
print("モデルの呼び出しと設定が完了しました。")
print(model)

モデルの呼び出しと設定が完了しました。
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)


In [ ]:
param_grid = {
    'n_estimators': [800,1000,1200],
    'learning_rate':[0.04,0.05,0.06],
    'max_depth': [5,6,7],
    'min_child_weight':[4,5,6],
    'subsample':[0.7,0.8,0.9]    
}
grid = GridSearchCV(xgb.XGBRegressor(), param_grid, cv=3, verbose=2, n_jobs=-1)
print("グリッドサーチのパラメータを設定しました")

In [ ]:
print("グリッドサーチを開始します")
grid.fit(train_x, train_y)

print("サーチが完了しました。ベストパラメータを出力します")
print(grid.best_params_)
model_grid = grid.best_params_

In [ ]:
# --- 交差検証（5-fold）　学習済みモデルの評価---
print("----------交差検証を開始します----------")
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# RMSE（scikit-learnは負の値で返すため符号反転）
rmse_scores = -cross_val_score(model, train_x, train_y, cv=cv, scoring='neg_root_mean_squared_error')
mae_scores  = -cross_val_score(model, train_x, train_y, cv=cv, scoring='neg_mean_absolute_error')
r2_scores   =  cross_val_score(model,train_x, train_y, cv=cv, scoring='r2')
print("----------スコアを掲載します----------")
print({
    "RMSE_mean": float(rmse_scores.mean()), "RMSE_std": float(rmse_scores.std()),
    "MAE_mean":  float(mae_scores.mean()),  "MAE_std":  float(mae_scores.std()),
    "R2_mean":   float(r2_scores.mean()),   "R2_std":   float(r2_scores.std())
})

----------交差検証を開始します----------


In [100]:
model.fit(train_x,train_y)
print("学習が完了しました。")

学習が完了しました。


In [101]:
y_pred = model.predict(test_x)
print("予測が完了しました。")

予測が完了しました。


In [102]:
sub = pd.DataFrame({
    "id": test_x["id"],
    "accident_risk": y_pred
})

print("予測データの作成が完了しました")
print(sub)

予測データの作成が完了しました
            id  accident_risk
0       517754       0.290799
1       517755       0.124976
2       517756       0.191029
3       517757       0.324557
4       517758       0.391344
...        ...            ...
172580  690334       0.112999
172581  690335       0.527779
172582  690336       0.269992
172583  690337       0.128276
172584  690338       0.482554

[172585 rows x 2 columns]


In [103]:
sub.to_csv("sub_xgb07.csv" , index=False)
print("CSVの出力が完了しました。Best_Public_Score:0.05585")

CSVの出力が完了しました。Best_Public_Score:0.05585


In [70]:
# プロファイルレポートの生成
profile = ProfileReport(train_df2, title='My Data Profiling Report')
profile.to_file("profile.html")
print("レポートの出力が完了しました。")

/usr/local/lib/python3.11/dist-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 23/23 [00:10<00:00,  2.13it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

レポートの出力が完了しました。
